### Detection of a sinusoidal signal in white Gaussian noise
_Øyvind Rørtveit, Naxys Technologies 2024_

We assume a signal of known frequency and duration, but unknown phase. A threshold is set on the absolute value of the inner product between the recorded noisy signal and a complex sinusoid at the same frequency. Using Neyman-Pearson detection theory, we can calculate the required threshold for a given false detection probability, based on the distribution of the statistic without presence of the signal (a $\Chi^2$-distribution with two degrees of freedom). The true positive rate can be found based on the threshold and the distribution of statistic in the presense of the signal (a non-central $\Chi^2$-distribution)


In [5]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interactive

def snr_threshold_given_false_positive_rate(false_positive_rate):
    """
    Compute the threhold in dB (above the noise level) needed
    to achieve the given false positive rate for a complex signal 
    with unknown phase in complex Gaussian noise
    """
    return 10*np.log10(sp.special.gammaincinv(1, 1-false_positive_rate))

def true_positive_rate_given_snr_and_threshold(threshold, snr, isPeak = False):
    """
    Compute the true positive rate for a complex signal 
    with unknown phase in complex Gaussian noise, using the given
    threshold and SNR.
    Set isPeal to true if the signal power it given in terms of peak power,
    set to false if it is given in terms of average power

    Uses the cumulative density function of the non-central Xi-square distribution with 2 degrees of freedom
    and non-centrality parameter given by 2*SNR

    """
    power_conversion_factor = 1.0
    if not isPeak:
        power_conversion_factor = 2.0
    return 1.0 - sp.stats.ncx2.cdf(2*(10**(threshold/10)), 2, power_conversion_factor*10**(snr/10))
    
    

def interactive_detection_plot(whitenoisedensity = 90, false_positives_per_year = 1.0, signal_duration_ms= 25):
    signal_duration = signal_duration_ms/1000
    num_recordings_per_year = 365*24*60*60/signal_duration
    false_positive_rate = false_positives_per_year/num_recordings_per_year
    threshold_db = snr_threshold_given_false_positive_rate(false_positive_rate)
    white_noise_power = whitenoisedensity - 10*np.log10(signal_duration)
    snr_range = np.arange(10, 20, 0.1)

    probability_of_detection = true_positive_rate_given_snr_and_threshold(threshold_db, snr_range)

    fig, ax = plt.subplots()
    ax.plot(snr_range, probability_of_detection)

    ax.set_xlabel("SNR (dB)")
    ax.set_ylabel("Probability of detection")
    ax2 = ax.twiny()
    ax2.set_xlim(snr_range[0]+white_noise_power, snr_range[-1]+white_noise_power)
    ax2.set_xlabel(r"Signal strength (dB ref $\mathrm{Pa}^2$)")


"""
JANUS FREQUENCY RANGES AND WAKEUP TONE DURATIONS:
A: 9.440 kHz - 13.600 kHz   Wakeup tone duration: 25 ms
B: 4.960 kHz - 7.040 kHz    Wakeup tone duration: 50 ms
C: 8.400 kHz - 11.000 kHz   Wakeup tone duration: 40 ms
D: 12.000 kHz - 16.160 kHz  Wakeup tone duration: 25 ms
E: 24.750 kHz - 31.250 kHz  Wakeup tone duration: 16 ms
"""

whitenoiseslider = ipywidgets.IntSlider(value = 90, min = 60, max = 140, step = 1, description = "White noise density (dB ref Pa^2/Hz)", style={'description_width': '230px'}, layout=ipywidgets.Layout(width='450px', justify_content='flex-end'))
fpa_slider = ipywidgets.FloatSlider(value = 1.0, min = 0.2, max = 1000, description = "False positives per year", style={'description_width': '230px'}, layout=ipywidgets.Layout(width='450px', justify_content='flex-end'))
duration_slider = ipywidgets.IntSlider(value = 25, min = 1, max = 50, step = 1, description = "Signal duration (ms)", style={'description_width': '230px'}, layout=ipywidgets.Layout(width='450px', justify_content='flex-end'))
interactive(interactive_detection_plot, whitenoisedensity = whitenoiseslider, false_positives_per_year = fpa_slider, signal_duration_ms = duration_slider)



interactive(children=(IntSlider(value=90, description='White noise density (dB ref Pa^2/Hz)', layout=Layout(ju…